In [2]:
#print("Create UDF 'create_import(s: str) -> str'")
!pip install openai


StatementMeta(, 2fb9c995-f52a-44af-8dea-1ff86a2214e2, 4, Finished, Available)

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *



df = spark.read.format("csv").option("header","true").option("sep", "|") \
                        .load("Files/Insurance_Claim_01.txt")
# df now is a Spark DataFrame containing CSV data from "Files/insuranceclaims.csv".
display(df)

StatementMeta(, 2fb9c995-f52a-44af-8dea-1ff86a2214e2, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, d712437f-2ef4-4237-9524-9bc988241746)

In [38]:
system_message = "Data Quality Assistant to fix data issues. Convert the date to the DD-MM-YYYY format. date: 01/12/24 converted date: 01-12-2024 ## date: 2002-05-01 converted date: 01-05-2002 ## date:2022-09-06 converted date:"
sample_text = '22.08.2022'
sample_response = '22-08-2022'
text = '2024/1/1'

openai.api_version = "2024-02-15-preview"
openai.api_base = "https://dqdemo01.openai.azure.com/"
openai.api_type = "azure"
openai.api_key = "14df843620644b0ab178768a855e0b79"

response = openai.ChatCompletion.create(
    engine="DQ-Demo03",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": sample_text},
        {"role": "assistant","content":  sample_response},

        {"role": "assistant","content": text }
    ]
)

print(response.choices[0].message['content'])


StatementMeta(, 2fb9c995-f52a-44af-8dea-1ff86a2214e2, 40, Finished, Available)

01-01-2024


In [24]:
openai.api_version = "2024-02-15-preview"
openai.api_base = "https://dqdemo01.openai.azure.com/"
openai.api_type = "azure"
openai.api_key = "14df843620644b0ab178768a855e0b79"

response = openai.ChatCompletion.create(
    engine = "DQ-Demo03",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": 'Extract zipcode, city, streetname & streetnumber from following address and put it in a JSON object structured as { "zipcode":"" , "city":"" , "streetname": "", "streetnumber":"" } Address:###High Street 27, EX4 3LJ Exeter ### '},
        {"role": "assistant","content":  ' { "zipcode":"EX4 3LJ" , "city":"Exeter" , "streetname": "High Street", "streetnumber":"27" }'},
        {"role": "assistant","content": 'Extract zipcode, city, streetname & streetnumber from following address and put it in a JSON object structured as { "zipcode":"" , "city":"" , "streetname": "", "streetnumber":"" } Address:###The Shambles, YO1 7LZ York ### json: ' }
    ]
)

print(response.choices[0].message['content'])

StatementMeta(, 2fb9c995-f52a-44af-8dea-1ff86a2214e2, 26, Finished, Available)

{ "zipcode":"YO1 7LZ" , "city":"York" , "streetname": "The Shambles", "streetnumber":"" }


In [39]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
import openai

def call_address_completion_api(prompt):
    try:

        openai.api_version = "2024-02-15-preview"
        openai.api_base = "https://dqdemo01.openai.azure.com/"
        openai.api_type = "azure"
        openai.api_key = "14df843620644b0ab178768a855e0b79"
        deployment_id = "DQ-Demo03"

        response = openai.ChatCompletion.create(
            engine="DQ-Demo03",
            messages=[
                {"role": "system", "content": 'Data Quality Assistant to fix data issues. Extract zipcode, city, streetname & streetnumber from following address and put it in a JSON object structured as { "zipcode":"" , "city":"" , "streetname": "", "streetnumber":"" } '},
                {"role": "user", "content": 'High Street 27, EX4 3LJ Exeter ### '},
                {"role": "assistant","content":  ' { "zipcode":"EX4 3LJ" , "city":"Exeter" , "streetname": "High Street", "streetnumber":"27" }'},
                {"role": "assistant","content": prompt }
            ]
        )

        #print(response.choices[0].message['content'])
        return response.choices[0].message['content']
    except Exception as e:
        print(f"Error in OpenAI API call: {e}")
        return e #"API call failed"


def call_date_completion_api(prompt):
    try:

        openai.api_version = "2024-02-15-preview"
        openai.api_base = "https://dqdemo01.openai.azure.com/"
        openai.api_type = "azure"
        openai.api_key = "14df843620644b0ab178768a855e0b79"
        deployment_id = "DQ-Demo03"

        response = openai.ChatCompletion.create(
            engine="DQ-Demo03",
            messages=[
                {"role": "system", "content": "Data Quality Assistant to fix data issues. can you guess date to the DD-MM-YYYY format from the given string, some examples follows. date: 01/12/24 converted date: 01-12-2024 ## date: 2002-05-01 converted date: 01-05-2002 ## date :2023/25/12 converted date: 25-12-2023## "},
                {"role": "user", "content": '22.08.2022'},
                {"role": "assistant","content":  ' 22-08-2022 '},
                {"role": "assistant","content": prompt }
            ]
        )

        #print(response.choices[0].message['content'])
        return response.choices[0].message['content']
    except Exception as e:
        print(f"Error in OpenAI API call: {e}")
        return e #"API call failed"

def call_country_completion_api(prompt):
    try:

        openai.api_version = "2024-02-15-preview"
        openai.api_base = "https://dqdemo01.openai.azure.com/"
        openai.api_type = "azure"
        openai.api_key = "14df843620644b0ab178768a855e0b79"
        deployment_id = "DQ-Demo03"

        response = openai.ChatCompletion.create(
            engine="DQ-Demo03",
            messages=[
                {"role": "system", "content": "Data Quality Assistant to fix data issues. Convert the country name to the ISO 3166-1 alpha-2 code and return only ISO code for the given country. if it already in ISO code and simply return same value with out any additional text/explanation "},
                {"role": "user", "content": 'België '},
                {"role": "assistant","content":  ' BE'},
                {"role": "assistant","content": prompt }
            ]
        )

        #print(response.choices[0].message['content'])
        return response.choices[0].message['content']
    except Exception as e:
        print(f"Error in OpenAI API call: {e}")
        return e #"API call failed"


def call_time_completion_api(prompt):
    try:

        openai.api_version = "2024-02-15-preview"
        openai.api_base = "https://dqdemo01.openai.azure.com/"
        openai.api_type = "azure"
        openai.api_key = "14df843620644b0ab178768a855e0b79"
        deployment_id = "DQ-Demo03"

        response = openai.ChatCompletion.create(
            engine="DQ-Demo03",
            messages=[
                {"role": "system", "content": "Data Quality Assistant to fix data issues. Convert the time to 24 hours. If the time is in 12hr notation, convert it to 24hr notation Time: 02:15 converted time: 02:15 ## Time: 02:15 p.m. convereted time: 14:15 ## return just only 24 hr time and if the time already in 24hr simply return same value without any text"},
                {"role": "user", "content": '1223 '},
                {"role": "assistant","content":  ' 12:23'},
                {"role": "user", "content": '04:24 '},
                {"role": "assistant","content":  ' 04:24'},
                {"role": "assistant","content": prompt }
            ]
        )

        #print(response.choices[0].message['content'])
        return response.choices[0].message['content']
    except Exception as e:
        print(f"Error in OpenAI API call: {e}")
        return e #"API call failed"

# Register the UDF
country_completion = udf(call_country_completion_api, StringType())
address_completion = udf(call_address_completion_api, StringType())
date_completion = udf(call_date_completion_api, StringType())
time_completion = udf(call_time_completion_api, StringType())

#dfprompt = dfprompt.withColumn("Res", country_completion(col("accidentcountry")))
df_final = df.withColumn("Time", time_completion(col("accidenttime")))
df_final = df_final.withColumn("Country", country_completion(col("accidentcountry")))
df_final = df_final.withColumn("Date", date_completion(col("accidentdate")))
df_final = df_final.withColumn("Address", address_completion(col("insuranceaddress")))
display(df_final)


StatementMeta(, 2fb9c995-f52a-44af-8dea-1ff86a2214e2, 41, Finished, Available)

SynapseWidget(Synapse.DataFrame, 382a6c7c-ddfa-4672-b52b-3c7000d274a3)